In [124]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import nltk
import re

In [125]:
# import nltk
# nltk.download()

### Stop words / Function words removing.

Data input :

In [126]:
NorthAmerica = pd.read_csv("rawdata/NorthAmerica_Trends.csv",sep = ",", header = 0)

#NorthAmerica.columns = ['time','tweet','hashtags','location']
NorthAmerica.columns = ['time','tweet','hashtags','place', 'location']
print len(NorthAmerica.index)
NorthAmerica = NorthAmerica.drop_duplicates(subset='tweet',keep='last') #remove the duplicate tweets
print len(NorthAmerica.index)


TwitterData = NorthAmerica
TwitterData.head()


64817
30268


,time,tweet,hashtags,place,location
1,Sat Dec 05 02:00:59 +0000 2015,Zack Greinke Inks 6 Year Deal With ArizonaDiam...,NaN,NaN,NaN
2,Sat Dec 05 02:00:59 +0000 2015,Reporters Allowed in Home of Shooting Suspects...,NaN,NaN,NaN
8,Sat Dec 05 02:00:59 +0000 2015,iMessage just autocorrected Greinke to Grinch ...,NaN,Los Angeles,NaN
10,Sat Dec 05 02:00:59 +0000 2015,I might be the only person that doesn't care t...,NaN,NaN,NaN
12,Sat Dec 05 02:00:59 +0000 2015,Killers deserve no respect! https://t.co/c5q0...,NaN,NaN,NaN


### Cleaning, Tokenization and Stemming

Remove stopwords :

we define a function for cleaning the tweets.

In [131]:
#stemming
from nltk.stem.snowball import SnowballStemmer
#from __future__ import print_function, unicode_literals
# Create p_stemmer of class PorterStemmer
s_stemmer = SnowballStemmer("english")

def filter_tweet (sample):
    stopped_tokens = [re.sub(r'(^https?:\/\/.*[\r\n]*)|(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)|(\n)|([^\w])|(RT)',
                '', word, flags=re.MULTILINE) for word in sample.split(" ") if word not in stopwords.words('english')] 
    filtered_sample = filter(None,stopped_tokens)
    filtered_sample = [s_stemmer.stem(i).lower() for i in filtered_sample] 
    #filtered_sample = [s_stemmer(i.lower()) for i in filtered_sample]
    filtered_sample = " ".join(filtered_sample)
    return filtered_sample



### Build the List of tweets :

In [132]:
ListTweets = [filter_tweet(tweet) for tweet in TwitterData.tweet.values]

### Test by sampling :

In [133]:
#sample
sample_index = 5
print TwitterData.tweet.values[sample_index]+"\n\n"
print ListTweets[sample_index]

RT @PoliticalGroove: The GOP vs women's right to their own bodies #TotallyAbsurdWars


the gop vs women right bodi


In [134]:

print filter_tweet(TwitterData.tweet.values[115])
print TwitterData.tweet.values[115]

best potus gif ever ht
RT @EricWolfson: BEST. POTUS. GIF. EVER.
#NationalCookieDay

RT @BuzzFeed #ThanksObama
https://t.co/efst1XcehU || h/t @TheObamaDiary #p2


In [135]:
#lower for hashtags
ListHashtags =[str(hashtag).lower() for hashtag in TwitterData.hashtags.values]  

### Save the cleaned data to a data frame

In [136]:
TwitterData_cleaned = pd.DataFrame({"tweet": ListTweets, 
                                   "hashtags": ListHashtags,
                                   "location": TwitterData.location})

In [137]:
TwitterData_cleaned.to_csv("cleaned data/NorthAmerica_Trends_cleaned.csv", sep = ",", header = True, index = False)

In [138]:
TwitterData_cleaned[0:10]

,hashtags,location,tweet
1,nan,NaN,zack greink ink 6 year deal with arizonadiamon...
2,nan,NaN,report allow home shoot suspect media crew got...
8,nan,NaN,imessag autocorrect greink grinch first time e...
10,nan,NaN,i might person doesnt care scott weiland die
12,nan,NaN,killer deserv respect
13,totallyabsurdwars,NaN,the gop vs women right bodi
14,artbasel,NaN,s biggest boldest best moment
15,dbacks,NaN,greink age 32 go make 343 million annual 6 yea...
17,theexperience10,NaN,this lord wow
18,dothan alabama drugs war huntsville,NaN,cop plant anoth reason 2 end drug https


# Trends Pre-processing 

We will save two pandas.dataframe to store the 'keywords' and 'hashtags' seprately.

In [70]:
Trends = pd.read_csv("rawdata/Trends.csv",sep = ",", header = 0)

Trends.head()

,createdAt,trend
0,2015-12-05T02:14:57Z,Greinke
1,2015-12-05T02:14:57Z,#TwitterAwakens
2,2015-12-05T02:14:57Z,#BlueNeighbourhood
3,2015-12-05T02:14:57Z,Robert Loggia
4,2015-12-05T02:14:57Z,#RDFamilyHoliday


In [71]:
ListTrends =[str(hashtag).lower() for hashtag in Trends.trend.values]
ListTrends[0:10]

['greinke',
 '#twitterawakens',
 '#blueneighbourhood',
 'robert loggia',
 '#rdfamilyholiday',
 '#undateablelive',
 'dodgers',
 '#antm',
 'diamondbacks',
 'joe johnson']

In [72]:
Hashtags_cleaned = pd.DataFrame({"hashtag": [word[1:] for word in ListTrends if "#" in word]})
Hashtags_cleaned = Hashtags_cleaned.drop_duplicates(subset='hashtag',keep='last')
print len(Hashtags_cleaned)
Hashtags_cleaned.head()

30


,hashtag
55,engagechat
85,twitterawakens
86,rdfamilyholiday
87,blueneighbourhood
88,antm


In [73]:
Keywords_cleaned = pd.DataFrame({"keyword": [word for word in ListTrends if "#" not in word]})
Keywords_cleaned = Keywords_cleaned.drop_duplicates(subset='keyword',keep='last')
print len(Keywords_cleaned)
Keywords_cleaned.head()

23


,keyword
43,tom holmoe
66,trend
67,greinke
68,robert loggia
69,dodgers


In [74]:
Hashtags_cleaned.to_csv("cleaned data/Hashtags_cleaned1205.csv", sep = ",", header = True, index = False)
Keywords_cleaned.to_csv("cleaned data/Keywords_cleaned1205.csv", sep = ",", header = True, index = False)